# ChurnSense: Decision Policy & Cost Simulation

## Why this notebook exists
The goal of ChurnSense is not perfect prediction — it is **better decisions under uncertainty**.

In Notebooks 1–3 we:
- explored the data and validated churn drivers (including SQL-based segmentation)
- built an interpretable churn risk model
- removed post-outcome leakage to ensure deployable performance

In this notebook we translate model scores into **operational retention actions**:
- Who should we intervene on?
- What does it cost?
- What value do we expect to retain?
- How do policies compare under budget constraints?

This notebook is the capstone: **prediction → decision → expected business impact**.

## Decision inputs and assumptions

We define a retention intervention (e.g., a $5 coupon, a phone call, or a service credit).
The decision problem is constrained by limited capacity/budget.

### Definitions
- **Risk score**: model-predicted churn probability, `p(churn)`
- **Intervention cost**: cost per contacted customer, `c`
- **Retention effect**: expected probability we prevent churn if we intervene, `u`
  - This is sometimes called *uplift* or *treatment effect*
- **Value at risk**: dollar value we retain if churn is prevented, `v`

### Expected value (per customer)
If we intervene on a customer with churn risk `p`, then the expected net value is:

\[
EV = (p \cdot u \cdot v) - c
\]

### Policy choices
We will evaluate two practical policies:
1. **Top-N policy**: intervene on the top N highest-risk customers (capacity constraint)
2. **Threshold policy**: intervene on customers with `p(churn) >= t`

We then compute:
- total campaign cost
- total expected value retained
- net expected value (value − cost)

## Where SQL fits (downstream, operational)

In Notebook 2, SQL was used for hypothesis validation (e.g., churn rate by Contract).
In this notebook, SQL is used **downstream** to operationalize decisions:

- rank customers by churn risk
- select customers under a budget (Top-N)
- compute aggregate cost and expected value
- produce a retention action table suitable for activation

This mirrors production workflows where the warehouse is used to generate
daily/weekly campaign lists and performance summaries.

In [1]:
import pandas as pd
import numpy as np

from pathlib import Path

# Modeling utilities
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Preprocessing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Model
from sklearn.linear_model import LogisticRegression

# Metrics
from sklearn.metrics import roc_auc_score, average_precision_score

# Optional: DuckDB for SQL over DataFrames / parquet
import duckdb

ARTIFACT_DIR = Path("artifacts")
ARTIFACT_DIR.mkdir(exist_ok=True, parents=True)

DATA_PATH = Path("..") / "data" / "raw" / "churn.csv"  # notebook lives in /notebooks
DATA_PATH

PosixPath('../data/raw/churn.csv')

In [2]:
# Load dataset
df = pd.read_csv(DATA_PATH)

# Keep an ID column for downstream activation (not used for modeling)
ID_COL = "CustomerID" if "CustomerID" in df.columns else None

df.shape, df.columns[:12], ID_COL

((7043, 50),
 Index(['CustomerID', 'Gender', 'Age', 'Under30', 'SeniorCitizen', 'Married',
        'Dependents', 'NumberofDependents', 'Country', 'State', 'City',
        'ZipCode'],
       dtype='object'),
 'CustomerID')

In [3]:
TARGET_COL = "ChurnCategory"

# Binary churn: 1 if churn reason exists, 0 if missing (NaN)
y = df[TARGET_COL].notna().astype(int)

# Keep ID for later, drop target from features
drop_cols = [TARGET_COL]
if ID_COL:
    drop_cols.append(ID_COL)

X = df.drop(columns=drop_cols)

y.value_counts(), y.mean(), X.shape

(ChurnCategory
 0    5174
 1    1869
 Name: count, dtype: int64,
 np.float64(0.2653698707936959),
 (7043, 48))

### Target definition (binary churn)

`ChurnCategory` is populated for customers who left (a reason is recorded) and missing for customers who stayed.
For decision policy design, we define a binary churn target:

- **1** = customer churned (any churn reason)
- **0** = customer did not churn

We keep `CustomerID` for downstream activation (campaign lists), but exclude it from modeling features.

In [4]:
# Explicitly remove post-churn / non-deployable columns
leakage_cols = [
    "ChurnLabel",
    "ChurnReason",
    "ChurnScore",
    "CustomerStatus",
    "TotalRefunds",
    "SatisfactionScore",
    "TotalRevenue",
    "CLTV",
    "Quarter",
    "Latitude",
    "Longitude",
    "ZipCode"
]

X_clean = X.drop(columns=[c for c in leakage_cols if c in X.columns])

X_clean.shape

(7043, 36)

### Leakage-safe feature set

For decision policy simulation, we restrict features to variables available **before** churn occurs.
Post-outcome signals (e.g., churn reason, satisfaction score, refunds, CLTV) are explicitly removed.

This ensures that risk scores reflect a realistic, deployable prediction scenario.

In [5]:
# Train/test split (stratified) for an honest performance check
X_train, X_test, y_train, y_test = train_test_split(
    X_clean, y, test_size=0.2, random_state=42, stratify=y
)

# Identify column types on the CLEAN feature set
cat_cols = X_train.select_dtypes(include=["object", "category", "bool"]).columns.tolist()
num_cols = [c for c in X_train.columns if c not in cat_cols]

# Preprocess + model (interpretable scoring)
preprocess = ColumnTransformer(
    transformers=[
        ("num", Pipeline([
            ("imputer", SimpleImputer(strategy="median")),
            ("scaler", StandardScaler())
        ]), num_cols),
        ("cat", Pipeline([
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("ohe", OneHotEncoder(handle_unknown="ignore"))
        ]), cat_cols),
    ],
    remainder="drop"
)

model = Pipeline(steps=[
    ("preprocess", preprocess),
    ("clf", LogisticRegression(max_iter=5000, solver="saga", n_jobs=-1))
])

model.fit(X_train, y_train)

# Quick sanity metrics (ranking quality)
proba_test = model.predict_proba(X_test)[:, 1]
roc = roc_auc_score(y_test, proba_test)
pr  = average_precision_score(y_test, proba_test)

roc, pr, len(num_cols), len(cat_cols)

(0.902482626779302, 0.7735283527568386, 11, 25)

### Customer-level risk scores

Using the leakage-safe logistic regression model, we score every customer with a predicted
churn probability (`p_churn`).

This table represents the **core output of the model** and is the only artifact required
for downstream decision-making.

Key points:
- Scores are **used for ranking**, not binary classification
- No probability threshold is applied at this stage
- Higher `p_churn` ⇒ higher priority for intervention
- `y_true` is included **only for evaluation and simulation**, not for real deployment

All subsequent policies operate on this scored customer table.

In [6]:
# Score all customers (used for policy decisions)
p_churn = model.predict_proba(X_clean)[:, 1]

scores = pd.DataFrame({
    "CustomerID": df[ID_COL].values,
    "p_churn": p_churn,
    "y_true": y.values  # for evaluation only (not used in real deployment)
})

scores.sort_values("p_churn", ascending=False).head(10)

,CustomerID,p_churn,y_true
20,2840-XANRC,0.990982,1
1620,8835-VSDSE,0.988779,1
126,4795-KTRTH,0.986983,1
2178,8821-XNHVZ,0.986442,1
132,9061-TIHDA,0.984696,1
2504,7384-GHBPI,0.984290,1
1212,0259-GBZSH,0.983989,1
357,7932-WPTDS,0.983977,1
443,3319-DWOEP,0.983901,1
1276,4952-YSOGZ,0.982244,1


### Expected Value (EV) decision rule

Rather than using an arbitrary probability threshold, we base intervention decisions on
expected economic value.

For each customer *i*, the expected value of intervening is:

EVᵢ = (pᵢ × uplift × value) − cost

Where:
- pᵢ is the predicted churn probability
- uplift is the probability that the intervention prevents churn
- value is the dollar value retained if churn is prevented
- cost is the cost of the intervention

We intervene **only when EVᵢ > 0**, ensuring that every action is expected to be profitable.

This rule translates model outputs into economically rational decisions under uncertainty.

### Policy assumptions

To operationalize decisions, we define the following baseline assumptions:

- **Intervention cost (c)** = $20  
  (e.g., coupon, service credit, or call center outreach)

- **Customer value (v)** = $500  
  (expected future value retained if churn is prevented)

- **Uplift (u)** = 0.20  
  (20% probability that the intervention prevents churn among customers who would churn)

These values are not treated as ground truth; they are scenario parameters that can be
adjusted without retraining the model.

In [7]:
# Policy parameters
uplift = 0.2
value = 500
cost = 20

# Expected value per customer
scores["EV"] = scores["p_churn"] * uplift * value - cost

scores.sort_values("EV", ascending=False).head(10)

,CustomerID,p_churn,y_true,EV
20,2840-XANRC,0.990982,1,79.098196
1620,8835-VSDSE,0.988779,1,78.877920
126,4795-KTRTH,0.986983,1,78.698254
2178,8821-XNHVZ,0.986442,1,78.644197
132,9061-TIHDA,0.984696,1,78.469573
2504,7384-GHBPI,0.984290,1,78.428974
1212,0259-GBZSH,0.983989,1,78.398922
357,7932-WPTDS,0.983977,1,78.397675
443,3319-DWOEP,0.983901,1,78.390060
1276,4952-YSOGZ,0.982244,1,78.224401


### EV-based intervention policy

We define an economically rational intervention rule:

- Intervene on a customer **only if EV > 0**
- Do not intervene if EV ≤ 0

This ensures that each action is expected to generate positive net value.
Unlike probability thresholds, this rule adapts naturally to changes in cost,
customer value, or intervention effectiveness.

### Capacity constraint (operational reality)

In practice, retention teams have limited outreach capacity (budget, agents, or campaign limits).
We therefore apply a two-step policy:

1. Keep only customers with **EV > 0**
2. From those, select the **Top-N** customers by expected value (or risk score)

This mirrors how campaigns are executed: profitable targets first, then capacity decides the cutoff.

In [8]:
# Capacity (number of customers we can contact)
N = 500

# Select profitable customers, then take Top-N by EV
eligible = scores[scores["EV"] > 0].copy()
selected = eligible.sort_values("EV", ascending=False).head(N)

n_eligible = len(eligible)
n_selected = len(selected)

# Campaign-level rollups (expected, not actual)
total_cost = n_selected * cost
total_expected_retained_value = (selected["p_churn"] * uplift * value).sum()
total_expected_net_value = selected["EV"].sum()

n_eligible, n_selected, total_cost, total_expected_retained_value, total_expected_net_value

(2989,
 500,
 10000,
 np.float64(45031.74453864222),
 np.float64(35031.74453864222))

### Campaign interpretation

The EV + capacity policy ensures that:
- Only customers with positive expected value are considered
- Capacity acts as an upper bound, not a target
- Campaign profitability is evaluated **in expectation**, not deterministically

A positive total expected net value indicates that the campaign is economically justified
under the stated assumptions, even though individual outcomes remain uncertain.

### SQL-based activation (production alignment)

In a production setting, model scores are written to a data warehouse.
SQL is then used to:

- Filter customers based on business rules (EV > 0)
- Rank customers by expected value
- Enforce capacity constraints (Top-N)
- Produce an activation table for downstream systems

This notebook mirrors that workflow by using SQL to operationalize
model outputs into concrete retention actions.

In [9]:
# Load scores into DuckDB for operational selection
con = duckdb.connect()

con.register("scores", scores)

query = f"""
SELECT
    CustomerID,
    p_churn,
    EV
FROM scores
WHERE EV > 0
ORDER BY EV DESC
LIMIT {N}
"""

activation_table = con.execute(query).df()
activation_table.head(), len(activation_table)

(   CustomerID   p_churn         EV
 0  2840-XANRC  0.990982  79.098196
 1  8835-VSDSE  0.988779  78.877920
 2  4795-KTRTH  0.986983  78.698254
 3  8821-XNHVZ  0.986442  78.644197
 4  9061-TIHDA  0.984696  78.469573,
 500)

In [10]:
con.register("activation_table", activation_table)

rollup_query = f"""
SELECT
    COUNT(*) AS targeted_customers,
    {cost} * COUNT(*) AS total_cost,
    SUM(p_churn * {uplift} * {value}) AS expected_retained_value,
    SUM(EV) AS expected_net_value,
    AVG(p_churn) AS avg_risk_in_target,
    MIN(p_churn) AS min_risk_in_target,
    MAX(p_churn) AS max_risk_in_target
FROM activation_table
"""

campaign_summary = con.execute(rollup_query).df()
campaign_summary

,targeted_customers,total_cost,expected_retained_value,expected_net_value,avg_risk_in_target,min_risk_in_target,max_risk_in_target
0,500,10000,45031.744539,35031.744539,0.900635,0.827599,0.990982


### Assumptions and risk considerations

A positive expected net value indicates the campaign is profitable **in expectation**.
However, realized outcomes depend on key assumptions:

- **Uplift accuracy**: if the true intervention effectiveness is lower than assumed,
  expected value will be overstated.
- **Customer value estimation**: if retained value is lower or delayed,
  realized ROI may fall short of expectations.
- **Operational execution**: timing, offer design, and customer response behavior
  can further impact outcomes.

These risks motivate sensitivity analysis and ongoing monitoring.

In [11]:
# Sensitivity analysis over uplift
uplift_grid = np.linspace(0.05, 0.5, 10)

sensitivity = []

for u in uplift_grid:
    ev = scores["p_churn"] * u * value - cost
    eligible = ev[ev > 0]
    selected_ev = eligible.sort_values(ascending=False).head(N)
    
    sensitivity.append({
        "uplift": u,
        "customers_targeted": len(selected_ev),
        "expected_net_value": selected_ev.sum()
    })

sensitivity_df = pd.DataFrame(sensitivity)
sensitivity_df

,uplift,customers_targeted,expected_net_value
0,0.05,500,1257.936135
1,0.10,500,12515.872269
2,0.15,500,23773.808404
3,0.20,500,35031.744539
4,0.25,500,46289.680673
5,0.30,500,57547.616808
6,0.35,500,68805.552943
7,0.40,500,80063.489077
8,0.45,500,91321.425212
9,0.50,500,102579.361347


### Sensitivity analysis takeaway

The campaign becomes meaningfully profitable at an uplift of approximately **0.15**.
Below this level, expected value is positive but fragile to estimation error.
Above this level, profitability scales quickly and remains robust.

This insight is more actionable than traditional model metrics (e.g., ROC-AUC),
because it identifies the **economic conditions under which the model should be used**.
Model quality enables decisions, but business assumptions determine value.

## Key Takeaways — Decision Policy & Cost Simulation

- Churn models should be evaluated by their **decision impact**, not just predictive accuracy.
- Expected value provides a principled framework for translating risk scores into actions.
- Capacity constraints naturally integrate with EV-based policies in real operations.
- SQL plays a critical downstream role in ranking, filtering, and activating customers.
- Sensitivity analysis reveals when a model is economically useful and when it is not.

Together, these steps transform churn prediction into a deployable,
economically grounded decision system.

## Decision Table for BI & Stakeholder Use

This section produces a single, decision-ready dataset that serves as the
source of truth for downstream dashboards and stakeholder decision-making.

All modeling and policy logic is resolved upstream. The output below is
intentionally flattened to support BI tools such as Power BI.

1) Set policy parameters (uplift, value, cost, K)

2) Ensure df has needed segmentation fields
   - Create tenure_band from TenureInMonths
   - Decide segmentation columns that actually exist (Contract, tenure_band, State, etc.)

3) Start from scores table (CustomerID, p_churn, y_true, EV already computed)
   - Add eligibility flag: EV > 0
   - Rank customers by EV (descending)
   - Add selected flag: eligible AND rank <= K
   - Add risk decile: decile of p_churn (optional but useful)

4) Build a segments table from df with only:
   - CustomerID + segmentation columns
   - drop duplicates

5) Merge decision_table with segments on CustomerID (left join)

6) Save decision_table to artifacts/churnsense_decision_table.csv

In [ ]:
# 1) Policy parameters
K = 500
uplift = 0.20
value = 500
cost = 20

# 2) Create tenure bands (because df doesn't have tenure_band yet)
df["tenure_band"] = pd.cut(
    df["TenureinMonths"],
    bins=[0, 12, 24, 48, 72, 10**9],
    labels=["0–1y", "1–2y", "2–4y", "4–6y", "6y+"],
    right=False
)

# 3) Choose segmentation columns that actually exist in df
segment_cols = [
    "Contract",
    "tenure_band",
    "State",
    "InternetService",
    "PaymentMethod",
]

In [ ]:
# ----- 4) Build decision-ready table from scores -----
decision_table = scores.copy()

# eligibility: expected net value per customer must be positive
decision_table["eligible"] = decision_table["EV"] > 0

# rank customers by EV (higher EV = higher priority)
decision_table["rank_by_ev"] = decision_table["EV"].rank(
    method="first",
    ascending=False
)

# selected = eligible AND within capacity K
decision_table["selected"] = decision_table["eligible"] & (decision_table["rank_by_ev"] <= K)

# optional: risk decile for distribution + reporting
decision_table["risk_decile"] = pd.qcut(
    decision_table["p_churn"],
    10,
    labels=[f"D{i}" for i in range(1, 11)]
)

# ----- 5) Attach segmentation columns from the original df -----
# Use ID_COL (CustomerID) and segment columns that exist
segments = df[[ID_COL] + segment_cols].drop_duplicates()

decision_table = decision_table.merge(
    segments,
    on=ID_COL,
    how="left"
)

# ----- 6) Save to artifacts -----
artifact_path = "../artifacts/churnsense_decision_table.csv"
decision_table.to_csv(artifact_path, index=False)

artifact_path, decision_table.shape, decision_table.head()

('../artifacts/churnsense_decision_table.csv',
 (7043, 13),
    CustomerID   p_churn  y_true         EV  eligible  rank_by_ev  selected  \
 0  8779-QRDMV  0.807586       1  60.758581      True       593.0     False   
 1  7495-OOKFY  0.771459       1  57.145941      True       725.0     False   
 2  1658-BYGOY  0.558580       1  35.857969      True      1489.0     False   
 3  4598-XLKNJ  0.468231       1  26.823121      True      1797.0     False   
 4  4846-WHAFZ  0.565033       1  36.503290      True      1478.0     False   
 
   risk_decile        Contract tenure_band       State InternetService  \
 0         D10  Month-to-Month        0–1y  California             Yes   
 1          D9  Month-to-Month        0–1y  California             Yes   
 2          D8  Month-to-Month        1–2y  California             Yes   
 3          D8  Month-to-Month        2–4y  California             Yes   
 4          D8  Month-to-Month        2–4y  California             Yes   
 
      PaymentMetho

In [14]:
decision_table.columns.tolist()

['CustomerID',
 'p_churn',
 'y_true',
 'EV',
 'eligible',
 'rank_by_ev',
 'selected',
 'risk_decile',
 'Contract',
 'tenure_band',
 'State',
 'InternetService',
 'PaymentMethod']